# Precificador Imobiliário

## Bibliotecas

In [9]:
import pandas as pd

## Carregar a Base

In [5]:
df = pd.read_csv('imoveis.csv')
df

,Título,Valor,area,Valor_metro,quartos,suites,garagem,iptu,condominio,regiao,...,SPA,Área de Serviço Coberta,Piso em Porcelanato,Vista Livre,Home Theater,Aceita Pet,Academia,tipo,Latitude,Longitude
0,SQNW 310 Bloco B,5100.0,76.0,67.105263,2,1,2,409,791,NOROESTE,...,0,0,0,0,0,0,0,Apartamento,-15.744342,-47.912500
1,SQNW 302,8500.0,69.0,123.188406,2,2,2,SemInformacao,SemInformacao,NOROESTE,...,0,0,0,1,0,0,0,Apartamento,-15.768450,-47.910646
2,SCRN 708/709,1620.0,31.0,52.258065,1,1,1,781,410,ASA NORTE,...,0,0,0,0,0,0,0,Kitnet,-15.763582,-47.892389
3,SQS 316,9000.0,240.0,37.500000,4,1,2,SemInformacao,2400,ASA SUL,...,0,0,0,1,0,0,0,Apartamento,-15.829897,-47.927249
4,SQNW 309 Bloco D,11000.0,155.0,70.967742,4,1,2,1319,1845,NOROESTE,...,0,0,0,0,0,0,0,Apartamento,-15.746442,-47.912663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58045,SHIS QI 7,12900000.0,650.0,19846.153846,4,4,4,SemInformacao,SemInformacao,LAGO SUL,...,0,0,0,0,0,0,0,Casa,-15.844132,-47.891195
58046,SHIS QL 10,18500000.0,850.0,21764.705882,5,5,6,SemInformacao,SemInformacao,LAGO SUL,...,0,0,0,0,0,0,0,Casa,-15.829325,-47.872777
58047,SHA Conjunto 5 Chacará 19A,1500000.0,500.0,3000.000000,4,3,3,SemInformacao,SemInformacao,ARNIQUEIRA,...,0,0,0,0,0,0,0,Casa,31.197678,121.334483
58048,Condomínio Santa Felicidade,1340000.0,250.0,5360.000000,3,3,2,SemInformacao,750,JARDIM BOTANICO,...,0,0,0,0,0,0,0,Casa,-15.572722,-47.333241


## Tratar a Base

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58050 entries, 0 to 58049
Data columns (total 58 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Título                   58050 non-null  object 
 1   Valor                    58050 non-null  float64
 2   area                     58050 non-null  float64
 3   Valor_metro              58050 non-null  float64
 4   quartos                  58050 non-null  int64  
 5   suites                   58050 non-null  int64  
 6   garagem                  58050 non-null  int64  
 7   iptu                     58050 non-null  object 
 8   condominio               58050 non-null  object 
 9   regiao                   58050 non-null  object 
 10  cidade                   58050 non-null  object 
 11  imobiliaria              58050 non-null  object 
 12  creci                    58050 non-null  object 
 13  operacao                 58050 non-null  object 
 14  Aquecimento Solar     

In [8]:
df.operacao.value_counts()

operacao
venda      57120
aluguel      930
Name: count, dtype: int64